# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps #!pip install gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Load file from weather challenge

city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.12,-134.97,78.03,79,0,16.96,PF,1579318852
1,1,mataura,-46.19,168.86,75.69,43,100,1.39,NZ,1579318889
2,2,albany,42.60,-73.97,12.20,51,40,10.29,US,1579318889
3,3,freeport,40.66,-73.58,24.80,34,1,8.05,US,1579318358
4,4,saint-pierre,-21.34,55.48,86.00,66,100,2.24,RE,1579318889


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Access gmaps with unique API key

gmaps.configure(api_key=g_key)

#example: gmaps.configure(api_key=gkey)

In [27]:
# Plot heatmap of humidity

fig = gmaps.figure()

# Create heat layer
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]


heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

#display figure
fig

# # # # IGNORE ALL THIS TO BOTTOM OF CELL

# # # In-class example

# # Plot Heatmap
# fig = gmaps.figure()

# # Create heat layer
# heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)

# # Add layer
# fig.add_layer(heat_layer)

# # Display figure
# fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# Narrow down the cities to fit weather conditions
# Instructions:
# - A max temperature lower than 80 degrees but higher than 70.
# - Wind speed less than 10 mph.
# - Zero cloudiness.
# - Drop any rows that don’t contain all three conditions. 

good_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)].dropna()
good_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,bubaque,11.28,-15.83,77.18,61,0,6.26,GW,1579318638
207,207,mankono,8.06,-6.19,72.81,29,0,2.64,CI,1579318676
262,262,mitu,1.20,-70.17,72.05,94,0,0.65,CO,1579318686
316,316,riachao das neves,-11.75,-44.91,71.37,82,0,1.41,BR,1579318696
409,409,bonthe,7.53,-12.51,77.07,88,0,0.96,SL,1579318962
413,413,mattru,7.62,-11.83,73.71,95,0,3.13,SL,1579318714
423,423,champerico,14.30,-91.92,76.08,79,0,5.06,GT,1579318716
440,440,ankazoabo,-22.28,44.52,71.42,85,0,0.40,MG,1579318719
471,471,kumphawapi,17.11,103.01,78.80,50,0,2.24,TH,1579318725
512,512,encarnacion,-27.33,-55.90,71.60,73,0,5.82,PY,1579318733


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Create dataframe to store hotel name

hotel_df = good_weather_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
10,bubaque,GW,11.28,-15.83,
207,mankono,CI,8.06,-6.19,
262,mitu,CO,1.20,-70.17,
316,riachao das neves,BR,-11.75,-44.91,
409,bonthe,SL,7.53,-12.51,
413,mattru,SL,7.62,-11.83,
423,champerico,GT,14.30,-91.92,
440,ankazoabo,MG,-22.28,44.52,
471,kumphawapi,TH,17.11,103.01,
512,encarnacion,PY,-27.33,-55.90,


In [32]:
# Set parameters to search for hotels with 500 meters

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use search term: hotel and out lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    name_address = requests.get(base_url, params=params).json()
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except(KeyError, IndexError): 
        print("Missing field/result...skipping")
hotel_df
    

Missing field/result...skipping
Missing field/result...skipping


,City,Country,Lat,Lng,Hotel Name
10,bubaque,GW,11.28,-15.83,Casa Dora
207,mankono,CI,8.06,-6.19,Hotel Cajoux
262,mitu,CO,1.20,-70.17,
316,riachao das neves,BR,-11.75,-44.91,Casa de vó Dina
409,bonthe,SL,7.53,-12.51,Bonthe Holiday Village
413,mattru,SL,7.62,-11.83,Boima Gbondo's Compound
423,champerico,GT,14.30,-91.92,"Hotel y restaurante ""El Diamante"""
440,ankazoabo,MG,-22.28,44.52,
471,kumphawapi,TH,17.11,103.01,โรงแรมมีเพลส กุมภวาปี
512,encarnacion,PY,-27.33,-55.90,Hotel Posadas


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))